In [1]:
import os
import numpy as np
import cv2
subjects = ['','jokowi','prabowo']

In [2]:
def prepare_training_data(data_folder_path):

    dirs = os.listdir("training-data")
 
    faces = []
    labels = []
 
    for dir_name in dirs:
        if not dir_name.startswith("s"):
            continue;

        label = int(dir_name.replace("s", ""))
        subject_dir_path = data_folder_path + "/" + dir_name
        subject_images_names = os.listdir(subject_dir_path)

        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue;

            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name

            #read image
            image = cv2.imread(image_path)

            #display an image window to show the image
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)

            #detect face
            face, rect = detect_face(image)
    
            if face is not None:
                faces.append(face)
                labels.append(label)
 
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
 
    return faces, labels


In [3]:
#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray scale as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow: Haar classifier
    face_cascade = cv2.CascadeClassifier('/Users/Alwi/Desktop/DIP/Fisher face/lbpcascade_frontalface.xml')
 
    #let's detect multiscale images(some images may be closer to camera than others)
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
 
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
 
    #under the assumption that there will be only one face,
    #extract the face area
    x, y, w, h = faces[0]
 
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [4]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
    #function to draw text on give image starting from
    #passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
 
#data will be in two lists of same size
#one list will contain all the faces
#and the other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")
 
#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

#face_recognizer = cv2.face.createLBPHFaceRecognizer()
#face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer = cv2.face.FisherFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

Preparing data...
Data prepared
Total faces:  16
Total labels:  16


error: OpenCV(3.4.3) /Users/travis/build/skvark/opencv-python/opencv_contrib/modules/face/src/fisher_faces.cpp:81: error: (-210:Unsupported format or combination of formats) In the Fisherfaces method all input samples (training images) must be of equal size! Expected 51076 pixels, but was 8100 pixels. in function 'train'


In [ ]:
def predict(test_img):
    #TESTING
    #test_img = cv2.imread(test_img) #gak perlu
    #make a copy of the image as we don't want to change original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)
    face = cv2.resize(face,(100, 100))
 
    #predict the image using our face recognizer 
    label= face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label[0]]
 
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    return img

In [ ]:
("Predicting images...")
 
#load test images
test_img1 = cv2.imread("test-data/pakde_3.jpg")
#test_img2 = cv2.imread("test-data/test2.jpg")
 
#perform a prediction
predicted_img1 = predict(test_img1)
#predicted_img2 = predict(test_img2)
print("Prediction complete")
 
#display both images
cv2.imshow(subjects[1], predicted_img1)
#cv2.imshow(subjects[2], predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()